In [1]:
import pandas
import sqlalchemy
print(pandas.__version__)

#from folium.plugins import MarkerCluster # for clustering the markers
import plotly.express as px
import plotly.graph_objects as go

# 
pandas.set_option('max_columns', None)

1.3.4


In [2]:
def build_postgres_conn_string (param:dict) -> str:
    return f'postgresql+psycopg2://{param["user"]}:{param["password"]}@{param["host"]}:{param["port"]}/{param["dbname"]}'

def create_postgres_engine (param:dict) -> sqlalchemy.engine.base.Engine:
    conString = build_postgres_conn_string(param)
    engine = sqlalchemy.create_engine(conString, pool_recycle=3600)
    return engine

def read_postgres_from_queryfile (sqlpath:str, postgresLogin:dict) -> pandas.DataFrame:
    engine = create_postgres_engine(postgresLogin)
    
    scriptFile = open(sqlpath,'r')
    script = scriptFile.read()
    df = pandas.read_sql(script, engine)

    return df

In [3]:
# Parameters to create a connection to the MaStR-postgreSQL DB
CONN_PARAM_DICT = {
    "host": "10.0.0.102",
    "dbname": "mastr",
    "user": "uba_user",
    "password": "UBAit2021!",
    "port": "5432"
}

bundesländer = {
    1400:"Brandenburg",
    1401:"Berlin",
    1402:"Baden-Würtenberg",
    1403:"Bayern",
    1404:"Bremen",
    1405:"Hessen",
    1406:"Hamburg",
    1407:"Mecklenburg-Vorpommern",
    1408:"Niedersachsen",
    1409:"Nordrhein-Westfahlen",
    1410:"Rheinland-Pfalz",
    1411:"Schleswig-Holstein",
    1412:"Saarland",
    1413:"Sachsen",
    1414:"Sachsen-Anhalt",
    1415:"Thüringen",
    1416:"Offshore"}

# Zubau von PV nach Bundesländern 

In [4]:
# Path to sql query files
SQL_PATH = r"./sqlQuerys/"
SQL_DATA = r"loadMastrSolar.sql"
SQL_DATA_PATH = SQL_PATH + SQL_DATA

dfMastrSolar = read_postgres_from_queryfile(sqlpath=SQL_DATA_PATH, postgresLogin=CONN_PARAM_DICT)
dfMastrSolar["Anlagenzahl"] = 1
dfMastrSolar = dfMastrSolar.replace({"Bundesland": bundesländer})
dfMastrSolar["Nettonennleistung"] = dfMastrSolar.loc[:,"Nettonennleistung"]/1000
dfMastrSolar2020 = dfMastrSolar[dfMastrSolar["Inbetriebnahmedatum"].dt.year == 2020]

In [5]:

dfMastrSolar2020 = dfMastrSolar2020.groupby(by=["Bundesland"]).sum()[["Anlagenzahl","Nettonennleistung"]]

dfMastrSolar2020

,Anlagenzahl,Nettonennleistung
Bundesland,,
Baden-Würtenberg,32583,560.394066
Bayern,47132,1212.418273
Berlin,1528,15.950649
Brandenburg,6483,383.897459
Bremen,303,3.830033
Hamburg,643,8.262660
Hessen,11701,201.506869
Mecklenburg-Vorpommern,2508,328.593352
Niedersachsen,17649,373.842565


# Zubau von Wind nach Bundesländern

In [6]:
# Path to sql query files
SQL_PATH = r"./sqlQuerys/"
SQL_DATA = r"loadMastrWind.sql"
SQL_DATA_PATH = SQL_PATH + SQL_DATA

dfMastrWind = read_postgres_from_queryfile(sqlpath=SQL_DATA_PATH, postgresLogin=CONN_PARAM_DICT)
dfMastrWind["Anlagenzahl"] = 1
dfMastrWind = dfMastrWind.replace({"Bundesland": bundesländer})
dfMastrWind["Nettonennleistung"] = dfMastrWind.loc[:,"Nettonennleistung"]/1000
dfMastrWind2020 = dfMastrWind[dfMastrWind["Inbetriebnahmedatum"].dt.year == 2020]

### Onshore

In [7]:
dfMastrWindOnshore = dfMastrWind2020[~dfMastrWind2020['Seelage'].notna()]
dfMastrWindOnshore = dfMastrWindOnshore.groupby(by=["Bundesland"]).sum()[["Anlagenzahl","Nettonennleistung"]]

dfMastrWindOnshore

,Anlagenzahl,Nettonennleistung
Bundesland,,
Baden-Würtenberg,16,37.25710
Bayern,11,31.71360
Brandenburg,72,242.26460
Hessen,26,82.06750
Mecklenburg-Vorpommern,33,100.43100
Niedersachsen,71,180.09800
Nordrhein-Westfahlen,91,310.55000
Rheinland-Pfalz,26,88.30000
Saarland,8,24.90000


### Offshore

In [8]:
dfMastrWindOffshore = dfMastrWind2020[dfMastrWind2020['Seelage'].notna()]
dfMastrWindOffshore = dfMastrWindOffshore.groupby(by=["Bundesland"]).sum()[["Anlagenzahl","Nettonennleistung"]]

dfMastrWindOffshore

,Anlagenzahl,Nettonennleistung
Bundesland,,
Niedersachsen,16,117.60
Offshore,16,101.28


# Biomasse

In [9]:
# Path to sql query files
SQL_PATH = r"./sqlQuerys/"
SQL_DATA = r"loadMastrBio.sql"
SQL_DATA_PATH = SQL_PATH + SQL_DATA

dfMastrBiomasse = read_postgres_from_queryfile(sqlpath=SQL_DATA_PATH, postgresLogin=CONN_PARAM_DICT)
dfMastrBiomasse["Anlagenzahl"] = 1
dfMastrBiomasse = dfMastrBiomasse.replace({"Bundesland": bundesländer})
dfMastrBiomasse["Nettonennleistung"] = dfMastrBiomasse.loc[:,"Nettonennleistung"]/1000
dfMastrBiomasse2020 = dfMastrBiomasse[dfMastrBiomasse["Inbetriebnahmedatum"].dt.year == 2020]

In [10]:
dfMastrBiomasse2020 = dfMastrBiomasse2020.groupby(by=["Bundesland"]).sum()[["Anlagenzahl","Nettonennleistung"]]

dfMastrBiomasse2020

,Anlagenzahl,Nettonennleistung
Bundesland,,
Baden-Würtenberg,121,44.59490
Bayern,244,89.02335
Berlin,2,0.05300
Brandenburg,21,16.42000
Hessen,28,12.58500
Mecklenburg-Vorpommern,17,14.51690
Niedersachsen,209,120.22500
Nordrhein-Westfahlen,108,51.63204
Rheinland-Pfalz,17,6.57100


# Zusammengefasst

In [13]:
dfs = {
    'Biomasse' : dfMastrBiomasse2020, 
    'Solar' : dfMastrSolar2020,
    'Wind_Onshore' : dfMastrWindOnshore,
    'Wind_Offshore' : dfMastrWindOffshore,
    }

pandas.concat(dfs.values(), axis=1, keys=dfs.keys()).round(2)

Biomasse                         Solar  \
                       Anlagenzahl Nettonennleistung Anlagenzahl   
Bundesland                                                         
Baden-Würtenberg             121.0             44.59     32583.0   
Bayern                       244.0             89.02     47132.0   
Berlin                         2.0              0.05      1528.0   
Brandenburg                   21.0             16.42      6483.0   
Hessen                        28.0             12.58     11701.0   
Mecklenburg-Vorpommern        17.0             14.52      2508.0   
Niedersachsen                209.0            120.22     17649.0   
Nordrhein-Westfahlen         108.0             51.63     34493.0   
Rheinland-Pfalz               17.0              6.57     10662.0   
Sachsen                       26.0             15.48      7031.0   
Sachsen-Anhalt                30.0             20.22      4722.0   
Schleswig-Holstein            44.0             38.52      4802.0   
Thüringen                      8.0              2.89      3490.0   
Bremen                         NaN               NaN       303.0   
Hamburg                        NaN               NaN       643.0   
Saarland                       NaN               NaN      1497.0   
Offshore                       NaN               NaN         NaN   

                                         Wind_Onshore                    \
                       Nettonennleistung  Anlagenzahl Nettonennleistung   
Bundesland                                                                
Baden-Würtenberg                  560.39         16.0             37.26   
Bayern                           1212.42         11.0             31.71   
Berlin                             15.95          NaN               NaN   
Brandenburg                       383.90         72.0            242.26   
Hessen                            201.51         26.0             82.07   
Mecklenburg-Vorpommern            328.59         33.0            100.43   
Niedersachsen                     373.84         71.0            180.10   
Nordrhein-Westfahlen              542.34         91.0            310.55   
Rheinland-Pfalz                   165.77         26.0             88.30   
Sachsen                           217.17          5.0              8.11   
Sachsen-Anhalt                    275.46         42.0            139.70   
Schleswig-Holstein                112.91         45.0            125.29   
Thüringen                         176.07         19.0             60.41   
Bremen                              3.83          NaN               NaN   
Hamburg                             8.26          NaN               NaN   
Saarland                           33.97          8.0             24.90   
Offshore                             NaN          NaN               NaN   

                       Wind_Offshore                    
                         Anlagenzahl Nettonennleistung  
Bundesland                                              
Baden-Würtenberg                 NaN               NaN  
Bayern                           NaN               NaN  
Berlin                           NaN               NaN  
Brandenburg                      NaN               NaN  
Hessen                           NaN               NaN  
Mecklenburg-Vorpommern           NaN               NaN  
Niedersachsen                   16.0            117.60  
Nordrhein-Westfahlen             NaN               NaN  
Rheinland-Pfalz                  NaN               NaN  
Sachsen                          NaN               NaN  
Sachsen-Anhalt                   NaN               NaN  
Schleswig-Holstein               NaN               NaN  
Thüringen                        NaN               NaN  
Bremen                           NaN               NaN  
Hamburg                          NaN               NaN  
Saarland                         NaN               NaN  
Offshore                        16.0            101.28